In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(ggpubr)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
currencies <- Currencies()

In [ ]:
startDate <- as.Date('2019-01-01')
currPair <- 'USDINR'
closestExpiries <- currencies$NseFuturesTimeSeries() %>%
    filter(TIME_STAMP >= startDate & SYMBOL = currPair) %>%
    group_by(TIME_STAMP) %>%
    summarize(EXPIRY_DT = min(EXPIRY)) %>%
    arrange(TIME_STAMP) 

nearestTimeSeries <- closestExpiries %>% 
    inner_join(currencies$NseFuturesTimeSeries(), by=c('TIME_STAMP', 'EXPIRY_DT' = 'EXPIRY')) %>%
    filter(TIME_STAMP >= startDate & SYMBOL = currPair) %>%
    select(TIME_STAMP.x, EXPIRY_DT, CLOSE, SETTLE) %>%
    arrange(TIME_STAMP)

nearestTimeSeries %>% print(n=100)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)